# Examples of split
A split agent has a single input stream and two or more output streams.

In [1]:
import os
import sys
sys.path.append("../")

In [2]:
from IoTPy.core.stream import Stream, run
from IoTPy.agent_types.split import split_element, split_list, split_window
from IoTPy.agent_types.split import unzip, separate, timed_unzip
from IoTPy.agent_types.basics import split_e, fsplit_2e
from IoTPy.helper_functions.recent_values import recent_values

## split_element
<b>split_element(func, in_stream, out_streams)</b>
<br>
<br>
where
<ol>
    <li><b>func</b> is a function with an argument which is an element of a single input stream and that returns a list with one element for each out_stream. <i>func</i> may have additional keyword arguments and may also have a state.</li>
    <li><b>in_stream</b> is a single input stream.</li>
    <li><b>out_streams</b> is a list of output streams.</li>
</ol>
In the example below, <i>func</i> is <i>f</i> which takes a single argument v (an element of the input stream) and returns a list of two values, one value for each of two output streams.
<br>
The agent split_element has a single input stream, <b>x</b> and a list <b>[y, z]</b> of output streams. The list of output streams correspond to the list of values returned by f. 
<br>
<br>
<b>y[n], z[n] = f(x[n])</b>
<br>
<br>
In this example, 
<br>
y[n] = x[n]+100 and z[n] = x[n]*2
<br>

#### Code
The code creates streams, x, y, and z, creates the split_element agent, and extends stream x. Calling run() executes a step in which all specified agents execute until all inputs have been processed. Then recent values of the output streams are printed.

In [3]:
def simple_example_of_split_element():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    def f(v): return [v+100, v*2]

    # Create agent with input stream x and output streams y, z.
    split_element(func=f, in_stream=x, out_streams=[y,z])
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))
    print ('Finished first run')
    
    # Put more test values in the input streams.
    x.extend(list(range(100, 105)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))
    print ('Finished second run.')

simple_example_of_split_element()

recent values of stream y are
[100, 101, 102, 103, 104]
recent values of stream z are
[0, 2, 4, 6, 8]
Finished first run
recent values of stream y are
[100, 101, 102, 103, 104, 200, 201, 202, 203, 204]
recent values of stream z are
[0, 2, 4, 6, 8, 200, 202, 204, 206, 208]
Finished second run.


### Using Lambda Expressions
Lambda expressions in split_element can be convenient as shown in this example which is essentially the same as the previous one.

In [4]:
def example_of_split_element_with_lambda():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Create agent with input stream x and output streams y, z.
    split_element(lambda v: [v+100, v*2], x, [y,z])
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))
example_of_split_element_with_lambda()

recent values of stream y are
[100, 101, 102, 103, 104]
recent values of stream z are
[0, 2, 4, 6, 8]


## Example of the decorator @split_e
The decorator <b>@split_e</b> operates the same as split_element, except that the agent is created by calling the decorated function.
<br>
Compare this example with the first example which used <i>split_element</i>. The two examples are almost identical. The difference is in the way that the agent is created. In this example, the agent is created by calling (the decorated) function <i>f</i> whereas in the previous example, the agent was created by calling <i>split_element</i>.

In [5]:
def simple_example_of_split_e():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    @split_e
    def f(v): return [v+100, v*2]

    # Create agent with input stream x and output streams y, z.
    f(in_stream=x, out_streams=[y,z])
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

simple_example_of_split_e()

recent values of stream y are
[100, 101, 102, 103, 104]
recent values of stream z are
[0, 2, 4, 6, 8]


## Example of functional forms
You may want to use a function that returns the streams resulting from a split instead of having the streams specified in out_streams, i.e. you may prefer to write:
<br>
<br>
a, b, c = h(u)
<br>
<br>
where <i>u</i> is a stream that is split into streams <i>a</i>, <i>b</i>, and <i>c</i>, 
instead of writing:
<br>
<br>
h(in_stream=u, out_streams=[a, b, c])
<br>
<br>
This example illustrates how a functional form can be specified and used. Function <i>h</i> creates and returns the three streams <i>x</i>, <i>y</i>, and <i>z</i>. Calling the function creates a <i>split_element</i> agent.

In [6]:
def simple_example_of_functional_form():

    # ------------------------------------------------------
    # Specifying a functional form
    # The functional form takes a single input stream and returns
    # three streams.
    def h(w):
        # Specify streams
        x = Stream('x')
        y = Stream('y')
        z = Stream('z')

        # Specify encapsulated functions
        def f(v): return [v+100, v*2, v**2]

        # Create agent with input stream x and output streams y, z.
        split_element(func=f, in_stream=w, out_streams=[x,y,z])

        # Return streams created by this function.
        return x, y, z
    # ------------------------------------------------------

    # Using the functional form.
    # Specify streams
    w = Stream('w')

    # Create agent with input stream x and output streams a, b, c.
    a, b, c = h(w)
    
    # Put test values in the input streams.
    w.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream a are')
    print (recent_values(a))
    print ('recent values of stream b are')
    print (recent_values(b))
    print ('recent values of stream c are')
    print (recent_values(c))

simple_example_of_functional_form()

recent values of stream a are
[100, 101, 102, 103, 104]
recent values of stream b are
[0, 2, 4, 6, 8]
recent values of stream c are
[0, 1, 4, 9, 16]


## Example with keyword arguments
This example shows how to pass keyword arguments to <i>split_element</i>. In the example, <i>addend</i> and <i>multiplicand</i> are arguments of <i>f</i> the encapsulated function, and these arguments are passed as keyword arguments to <i>split_element</i>.

In [7]:
def example_of_split_element_with_keyword_args():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    def f(v, addend, multiplicand): 
        return [v+addend, v*multiplicand]

    # Create agent with input stream x and output streams y, z.
    split_element(func=f, in_stream=x, out_streams=[y,z], addend=100, multiplicand=2)
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

example_of_split_element_with_keyword_args()

recent values of stream y are
[100, 101, 102, 103, 104]
recent values of stream z are
[0, 2, 4, 6, 8]


## Split element with state
This example shows how to create an agent with state. The encapsulated function takes two arguments --- an element of the input stream and a <b>state</b> --- and it returns two values: a list of elements corresponding to the output streams and the <b>next state</b>. The function may have additional arguments which are passed as keyword arguments to <i>split_element</i>.
<br>
<br>
The call <i>split_element(...)</i> to create the agent must have a keyword argument called <b>state</b> with its initial value. For example:
<br>
split_element(func=f, in_stream=x, out_streams=[y,z], <b>state=0</b>)  
<br>
In this example, the sequence of values of <i>state</i> is 0, 1, 2, .... which is also the sequence of values of the input stream and hence also of <i>v</i>.

In [8]:
def example_of_split_element_with_state():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    def f(v, state):
        next_state = state+1
        return ([v+state, v*state], next_state)

    # Create agent with input stream x and output streams y, z.
    split_element(func=f, in_stream=x, out_streams=[y,z], state=0)
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

example_of_split_element_with_state()

recent values of stream y are
[0, 2, 4, 6, 8]
recent values of stream z are
[0, 1, 4, 9, 16]


## Example with state and keyword arguments
This example shows an encapsulated function with a state and an argument called <i>state_increment</i> which is passed as a keyword argument to <i>split_element</i>.

In [9]:
def example_of_split_element_with_state_and_keyword_args():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    def f(v, state, state_increment):
        next_state = state + state_increment
        return ([v+state, v*state], next_state)

    # Create agent with input stream x and output streams y, z.
    split_element(func=f, in_stream=x, out_streams=[y,z], state=0, state_increment=10)
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

example_of_split_element_with_state_and_keyword_args()

recent values of stream y are
[0, 11, 22, 33, 44]
recent values of stream z are
[0, 10, 40, 90, 160]


## Example with StreamArray and NumPy arrays

In [10]:
import numpy as np
from IoTPy.core.stream import StreamArray

def example_of_split_element_with_stream_array():
    # Specify streams
    x = StreamArray('x')
    y = StreamArray('y')
    z = StreamArray('z')

    # Specify encapsulated functions
    def f(v, addend, multiplier):
        return [v+addend, v*multiplier]

    # Create agent with input stream x and output streams y, z.
    split_element(func=f, in_stream=x, out_streams=[y,z],
                 addend=1.0, multiplier=2.0)
    
    # Put test values in the input streams.
    A = np.linspace(0.0, 4.0, 5)
    x.extend(A)

    # Execute a step
    run()

    # Look at recent values of streams.
    assert np.array_equal(recent_values(y), A + 1.0)
    assert np.array_equal(recent_values(z), A * 2.0)
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

example_of_split_element_with_stream_array()

recent values of stream y are
[1. 2. 3. 4. 5.]
recent values of stream z are
[0. 2. 4. 6. 8.]


## Example of split list
split_list is the same as split_element except that the encapsulated function operates on a <i>list</i> of elements of the input stream rather than on a single element. Operating on a list can be more efficient than operating sequentially on each of the elements of the list. This is especially important when working with arrays.
<br>
<br>
In this example, f operates on a list, <i>lst</i> of elements, and has keyword arguments <i>addend</i> and <i>multiplier</i>. It returns two lists corresponding to two output streams of the agent.

In [11]:
def example_of_split_list():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    def f(lst, addend, multiplier):
        return ([v+addend for v in lst], [v*multiplier for v in lst])

    # Create agent with input stream x and output streams y, z.
    split_list(func=f, in_stream=x, out_streams=[y,z], addend=100, multiplier=2)
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

example_of_split_list()

recent values of stream y are
[100, 101, 102, 103, 104]
recent values of stream z are
[0, 2, 4, 6, 8]


## Example of split list with arrays
In this example, the encapsulated function <i>f</i> operates on an array <i>a</i> which is a segment of the input stream array, <i>x</i>. The operations in <i>f</i> are array operations (not list operations). For example, the result of <i>a * multiplier </i> is specified by numpy multiplication of an array with a scalar.

In [12]:
def example_of_split_list_with_arrays():
    # Specify streams
    x = StreamArray('x')
    y = StreamArray('y')
    z = StreamArray('z')

    # Specify encapsulated functions
    def f(a, addend, multiplier):
        # a is an array
        # return two arrays.
        return (a + addend, a * multiplier)

    # Create agent with input stream x and output streams y, z.
    split_list(func=f, in_stream=x, out_streams=[y,z], addend=100, multiplier=2)
    
    # Put test values in the input streams.
    x.extend(np.arange(5.0))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

example_of_split_list_with_arrays()

recent values of stream y are
[100. 101. 102. 103. 104.]
recent values of stream z are
[0. 2. 4. 6. 8.]


## Test of unzip
unzip is the opposite of zip_stream.
<br>
<br>
An element of the input stream is a list or tuple whose length is the same as the number of output streams; the <i>j</i>-th element of the list is placed in the <i>j</i>-th output stream.
<br>
<br>
In this example, when the unzip agent receives the triple (1, 10, 100) on the input stream <i>w</i> it puts 1 on stream <i>x</i>, and 10 on stream <i>y</i>, and 100 on stream <i>z</i>.

In [13]:
def simple_test_unzip():
    # Specify streams
    w = Stream('w')
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Create agent with input stream x and output streams y, z.
    unzip(in_stream=w, out_streams=[x,y,z])
    
    # Put test values in the input streams.
    w.extend([(1, 10, 100), (2, 20, 200), (3, 30, 300)])

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream x are')
    print (recent_values(x))
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

simple_test_unzip()

recent values of stream x are
[1, 2, 3]
recent values of stream y are
[10, 20, 30]
recent values of stream z are
[100, 200, 300]


## Example of separate
<b>separate</b> is the opposite of <b>mix</b>.
<br>
The elements of the input stream are pairs (index, value). When a pair <i>(i,v)</i> arrives on the input stream the value <i>v</i> is appended to the <i>i</i>-th output stream.
<br>
<br>
In this example, when (0, 1) and (2, 100) arrive on the input stream <i>x</i>, the value 1 is appended to the 0-th output stream which is <i>y</i> and the value 100 is appended to output stream indexed 2 which is stream <i>w</i>.

In [14]:
def simple_test_separate():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')
    w = Stream('w')

    # Create agent with input stream x and output streams y, z.
    separate(in_stream=x, out_streams=[y,z,w])
    
    # Put test values in the input streams.
    x.extend([(0,1), (2, 100), (0, 2), (1, 10), (1, 20)])

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))
    print ('recent values of stream w are')
    print (recent_values(w))

simple_test_separate()

recent values of stream y are
[1, 2]
recent values of stream z are
[10, 20]
recent values of stream w are
[100]


## Example of separate with stream arrays.
This is the same example as the previous case. The only difference is that since the elements of the input stream are pairs, the dimension of <i>x</i> is 2.

In [15]:
def test_separate_with_stream_array():
    # Specify streams
    x = StreamArray('x', dimension=2)
    y = StreamArray('y')
    z = StreamArray('z')

    # Create agent with input stream x and output streams y, z.
    separate(in_stream=x, out_streams=[y,z])
    
    # Put test values in the input streams.
    x.extend(np.array([[1.0, 10.0], [0.0, 2.0], [1.0, 20.0], [0.0, 4.0]]))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

test_separate_with_stream_array()

recent values of stream y are
[2. 4.]
recent values of stream z are
[10. 20.]


## Example of split window
The input stream is broken up into windows. In this example, with <i>window_size</i>=2 and <i>step_size</i>=2, the sequence of windows are <i>x[0, 1], x[2, 3], x[4, 5], ....</i>.
<br>
<br>
The encapsulated function operates on a window and returns <i>n</i> values where <i>n</i> is the number of output streams. In this example, max(window) is appended to the output stream with index 0, i.e. stream <i>y</i>, and min(window) is appended to the output stream with index 1, i.e., stream <i>z</i>.
<br>
<br>
Note: You can also use the lambda function as in:
<br>
split_window(lambda window: (max(window), min(window)), x, [y,z], 2, 2)

In [16]:
def simple_example_of_split_window():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Specify encapsulated functions
    def f(window): return (max(window), min(window))

    # Create agent with input stream x and output streams y, z.
    split_window(func=f, in_stream=x, out_streams=[y,z],
                window_size=2, step_size=2)
    
    # Put test values in the input streams.
    x.extend(list(range(5)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

simple_example_of_split_window()

recent values of stream y are
[1, 3]
recent values of stream z are
[0, 2]


## Example that illustrates zip followed by unzip is the identity.
zip_stream followed by unzip returns the initial streams.

In [17]:
from IoTPy.agent_types.merge import zip_stream
def example_zip_plus_unzip():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')
    u = Stream('u')
    v = Stream('v')

    # Create agents
    zip_stream(in_streams=[x,y], out_stream=z)
    unzip(in_stream=z, out_streams=[u,v])
    
    # Put test values in the input streams.
    x.extend(['A', 'B', 'C'])
    y.extend(list(range(100, 1000, 100)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream u are')
    print (recent_values(u))
    print ('recent values of stream v are')
    print (recent_values(v))

example_zip_plus_unzip()

recent values of stream u are
['A', 'B', 'C']
recent values of stream v are
[100, 200, 300]


## Example that illustrates that mix followed by separate is the identity.

In [18]:
from IoTPy.agent_types.merge import mix
def example_mix_plus_separate():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')
    u = Stream('u')
    v = Stream('v')

    # Create agents
    mix(in_streams=[x,y], out_stream=z)
    separate(in_stream=z, out_streams=[u,v])
    
    # Put test values in the input streams.
    x.extend(['A', 'B', 'C'])
    y.extend(list(range(100, 1000, 100)))

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream u are')
    print (recent_values(u))
    print ('recent values of stream v are')
    print (recent_values(v))

example_mix_plus_separate()

recent values of stream u are
['A', 'B', 'C']
recent values of stream v are
[100, 200, 300, 400, 500, 600, 700, 800, 900]


## Simple example of timed_unzip
An element of the input stream is a pair (timestamp, list). The sequence of timestamps must be increasing. The list has length n where n is the number of output streams. The m-th element of the list is the value of the m-th output stream associated with that timestamp. For example, if an element of the input stream <i>x</i> is (5, ["B", "a"]) then (5, "B") is appended to stream <i>y</i> and (5, "a') is appended to stream <i>z</i>.

In [19]:
def test_timed_unzip():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')

    # Create agent with input stream x and output streams y, z.
    timed_unzip(in_stream=x, out_streams=[y,z])
    
    # Put test values in the input streams.
    x.extend([(1, ["A", None]), (5, ["B", "a"]), (7, [None, "b"]),
              (9, ["C", "c"]), (10, [None, "d"])])

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))
    print ('recent values of stream z are')
    print (recent_values(z))

test_timed_unzip()

recent values of stream y are
[(1, 'A'), (5, 'B'), (9, 'C')]
recent values of stream z are
[(5, 'a'), (7, 'b'), (9, 'c'), (10, 'd')]


## Example that illustrates that timed_zip followed by timed_unzip is the identity.

In [20]:
from IoTPy.agent_types.merge import timed_zip
def test_timed_zip_plus_timed_unzip():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    z = Stream('z')
    u = Stream('u')
    v = Stream('v')

    # Create agents
    timed_zip(in_streams=[x,y], out_stream=z)
    timed_unzip(in_stream=z, out_streams=[u,v])
    
    # Put test values in the input streams.
    x.extend([[1, 'a'], [3, 'b'], [10, 'd'], [15, 'e'], [17, 'f']])
    y.extend([[2, 'A'], [3, 'B'], [9, 'D'], [20, 'E']])

    # Execute a step
    run()

    # Look at recent values of streams.
    print ('recent values of stream u are')
    print (recent_values(u))
    print ('recent values of stream v are')
    print (recent_values(v))

test_timed_zip_plus_timed_unzip()

recent values of stream u are
[(1, 'a'), (3, 'b'), (10, 'd'), (15, 'e'), (17, 'f')]
recent values of stream v are
[(2, 'A'), (3, 'B'), (9, 'D')]
